# Introdução

O objetivo desse programa é procurar por disciplinas do catálogo da UFABC e identificar disciplinas similares.

Escrito por: Marcelo Bussotti Reyes - CMCC - UFABC
Setembro de 2016

In [1]:
import string
import csv
import numpy as np
import time

Primeiramente obtitve o catálogo de disciplinas em formato excel, gentilmente fornecido pela Prof. Paula Tiba e sua equipe da Pró-Reitoria de Graduação. Exportei para formato csv, colocando como delimitador de campo "tab". O nome do arquivo é 

In [2]:
filename = 'catalogo2015.csv'
colSigla  = 0                     # coluna que contém as siglas das disciplinas
colNome   = 1                     # coluna com o nome das disciplinas
colEmenta = 4                     # coluna com as ementas
colBibliB = 5                     # coluna bibliografia básica
colBibliC = 6                     # coluna bibliografia complementar

stopWords = ['a'   , 'e' , 'é', 'o' , 'as' , 'os' ,'ao','aos', \
             'da'  , 'de', 'do' , 'das', 'dos',            \
             'em'  , 'na', 'no' , 'nos',                   \
             'para','com', 'por', 'à'  , 'às' , 'sobre',   \
             'um'   ,'uma',  'como', 'entre', 'que', 'ou',  \
             '¿'    , ]

ELIM_MOST_FREQ = 200               # além das palavras acima, esta opção permite 
                                   # eliminar palavras mais frequentes presentes nas ementas
ELIM_MULT_OCORRENCIAS = bool(1)    # se True - elimina a contagem múltipla de palavras, contanto somente 1 ocorrência

Para a identificação das disciplinas, compilamos todas as palavras de cada ementa e colocamos em um dicionário onde a chave é a palavra e o valor é o número de ocorrências da palavra na ementa. 

In [3]:
def sortFreqDict(freqdict):
    aux = [(freqdict[key], key) for key in freqdict]
    aux.sort()
    aux.reverse()
    return aux

In [4]:
# removendo as palavras muito frequentes como artigos e preposições 
# as stopWords foram definidas no início da rotina
def removeStopWords(texto,stopWords):
    for sw in stopWords:                           # Laço para cada stopWord
        # Remove as stopWords uma a uma. Foram incluídos espaços para evitar 
        #remover partes das palavras
        texto = texto.replace(' '+sw+' ',' ')
        #texto = texto.replace(sw+' ',' ')      
    return texto

In [5]:
def limpaTexto(texto,stopWords):
    #transformacao = str.maketrans('', '', string.punctuation) #creates a table for translation for all puntuation
    spcs = len(string.punctuation)*' '                        # creates a string with same length of punctuation
    transformacao = str.maketrans(string.punctuation,spcs)    # creates a map from punctuation to white spaces
    texto = texto.translate(transformacao)                    # removes all the punctuation
    texto = texto.lower()                                     # makes all words in lower case
    texto = removeStopWords(texto,stopWords)                  # removes stop words defined at the beginning
    texto = texto.replace('\n',' ')                           # removes the newline marks
    return texto

In [6]:
def criaVetor(texto):
    palavras = texto.split()                              # quebra a string em uma lista de palavras
    contagemPalavras = []                                 # inicia lista de palavras
    for w in palavras:                                    # loop para cada palavra
        contagemPalavras.append(palavras.count(w))        # conta o número de vezes que cada palavra ocorre na lista
                                                          # e acrescenta à lista contagemPalavras
    vetor =  dict(zip(palavras, contagemPalavras))        # Cria dicionário com as palavras e as respectivas contagens
    return vetor

In [7]:
catalogo = list(csv.reader(open(filename, 'r'), delimiter='\t'))

# Juntando todas as ementas

Aqui juntamos o texto de todas as ementas e colocamos num único string, para saber todas as palavras usadas

In [8]:
todasEmentas=''
len(catalogo)
for k in range(1,len(catalogo)):
        todasEmentas = todasEmentas + ' ' + catalogo[k][colEmenta] + ' ' + catalogo[k][colBibliB]
#todasEmentas = todasEmentas.replace('\n',' ')

# Removendo stop-words
Aqui, removemos as pelavras muito frequentes e que não têm a ver com a disciplina em si, mas com a estrutura do português, o que atrapalha no momento de quantificar a sobreposição entre as disciplinas. Usa a função criada no início do código chamada limpaTexto. Abaixo do código, tem um exemplo de texto após a 

In [9]:
print(todasEmentas[1:1000])

Contextualização histórica da emergência das teorias de relações internacionais. Matrizes filosóficas. Realismo. Idealismo. Debate clássico realismo versus idealismo. Escola inglesa. Behaviorismo. Debate metodológico. Neoliberalismo. Neorrealismo. Teoria da interdependência complexa. Neoinstitucionalismo. Cooperação e conflito. Sociedade internacional, anarquia. Regimes internacionais. CARR, Edward Hallett. Vinte anos de crise 1919-1939: uma introdução ao estudo das Relações Internacionais. São Paulo; Brasília: Imprensa Oficial do Estado; Ed. UNB, 2001.
KEOHANE, Robert O.; NYE, Joseph S. Power and Interdependence. Glenview: Scott Foresman, 1989.
KRASNER, Stephen (Ed). International Regimes. Ithaca; London: Cornell University Press, 1983.
MORGENTHAU, Hans J. A Política entre as Nações: a luta pelo poder e pela paz. São Paulo; Brasília: Imprensa Oficial do Estado; Ed. UNB, 2003.
WALTZ, Kenneth N. Teoria das Relações Internacionais. Lisboa: Gradiva, 2002. Introdução aos sistemas de acion


Abaixo segue um trecho do texto após a remoção das palavras frequentes, pontuação, etc.

In [10]:
# removendo pontuações e stop-words
todasEmentasLimpo = limpaTexto(todasEmentas,stopWords)
print(todasEmentasLimpo[1:1000])

contextualização histórica emergência teorias relações internacionais  matrizes filosóficas  realismo  idealismo  debate clássico realismo versus idealismo  escola inglesa  behaviorismo  debate metodológico  neoliberalismo  neorrealismo  teoria interdependência complexa  neoinstitucionalismo  cooperação conflito  sociedade internacional  anarquia  regimes internacionais  carr  edward hallett  vinte anos crise 1919 1939  introdução estudo relações internacionais  são paulo  brasília  imprensa oficial estado  ed  unb  2001  keohane  robert   nye  joseph s  power and interdependence  glenview  scott foresman  1989  krasner  stephen  ed   international regimes  ithaca  london  cornell university press  1983  morgenthau  hans j  política nações  luta pelo poder pela paz  são paulo  brasília  imprensa oficial estado  ed  unb  2003  waltz  kenneth n  teoria relações internacionais  lisboa  gradiva  2002  introdução sistemas acionamentos elétricos  elementos sistema acionamento elétrico  pont


## Este passo pode ser bastante demorado

Nesse ponto temos uma lista (palavras) e uma lista de quantas vezes cada palavra ocorre (contagemPalavras). Vamos agora criar um dicionário com esses pares, e ordená-lo da mais frequente para a menos frequente. 

!!!Bastante demorado!!!! 
pode levar até 5 minutos para rodar.

In [11]:
allPairs  = criaVetor(todasEmentasLimpo)
sortPairs = sortFreqDict(allPairs)                    # usa a função definida no início para ordenar em ordem decrescente

In [12]:
emptyPairs = {}                                       # inicia variável
for aux in allPairs.keys():                          # loop para todas as palavras
    emptyPairs[aux] = 0                              # cria um dicionário com todas as palavras, mas com contagem zero

Por curiosidade, vamos visualizar as palavras mais frequentes

In [13]:
for k in range(ELIM_MOST_FREQ+1):
    # just for visualization, let's see the mostr frequent words...
    print(str(k+1) + ": " + str(sortPairs[k][1]) + ' ==> '+ str(sortPairs[k][0]) + ' vezes')
    

1: p ==> 1630 vezes
2: ed ==> 1202 vezes
3: paulo ==> 1058 vezes
4: são ==> 1044 vezes
5: j ==> 613 vezes
6: rio ==> 534 vezes
7: m ==> 533 vezes
8: janeiro ==> 527 vezes
9: and ==> 508 vezes
10: sistemas ==> 483 vezes
11: r ==> 460 vezes
12: 2 ==> 440 vezes
13: editora ==> 415 vezes
14: c ==> 393 vezes
15: introdução ==> 339 vezes
16: s ==> 331 vezes
17: análise ==> 331 vezes
18: of ==> 329 vezes
19: l ==> 326 vezes
20: 2008 ==> 320 vezes
21: 2006 ==> 313 vezes
22: d ==> 312 vezes
23: teoria ==> 285 vezes
24: brasil ==> 281 vezes
25: desenvolvimento ==> 279 vezes
26: new ==> 273 vezes
27: h ==> 269 vezes
28: 1 ==> 262 vezes
29: 2007 ==> 257 vezes
30: f ==> 243 vezes
31: 3 ==> 241 vezes
32: 2005 ==> 240 vezes
33: 2009 ==> 234 vezes
34: york ==> 231 vezes
35: v ==> 228 vezes
36: g ==> 224 vezes
37: press ==> 222 vezes
38: isbn ==> 218 vezes
39: energia ==> 218 vezes
40: fundamentos ==> 215 vezes
41: aplicações ==> 215 vezes
42: 2004 ==> 214 vezes
43: conceitos ==> 209 vezes
44: engenhar

In [14]:
for k in range(ELIM_MOST_FREQ):               # loop for the number of words to be eliminated
    sortPairs.remove(sortPairs[0])            # elimina primeiro da lista e retorna o vetor "truncado"
    if k%10==0:
        print(k) 

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190


In [15]:
for k in range(ELIM_MOST_FREQ):
    # just for visualization, let's see the mostr frequent words...
    print(str(k) + ": " + str(sortPairs[k][1]) + ' ==> '+ str(sortPairs[k][0]) + ' vezes')

0: addison ==> 67 vezes
1: 2012 ==> 67 vezes
2: modelagem ==> 66 vezes
3: 1997 ==> 66 vezes
4: vol ==> 65 vezes
5: tempo ==> 65 vezes
6: robert ==> 65 vezes
7: coleção ==> 65 vezes
8: ser ==> 64 vezes
9: programação ==> 64 vezes
10: normas ==> 64 vezes
11: linguagem ==> 64 vezes
12: lei ==> 64 vezes
13: internacionais ==> 64 vezes
14: definição ==> 64 vezes
15: científica ==> 64 vezes
16: transferência ==> 62 vezes
17: temas ==> 62 vezes
18: guanabara ==> 62 vezes
19: comunicação ==> 62 vezes
20: ambientais ==> 62 vezes
21: 1996 ==> 62 vezes
22: uso ==> 61 vezes
23: universidade ==> 61 vezes
24: século ==> 61 vezes
25: práticas ==> 61 vezes
26: potência ==> 61 vezes
27: industrial ==> 61 vezes
28: espaço ==> 61 vezes
29: design ==> 61 vezes
30: atividade ==> 61 vezes
31: analysis ==> 61 vezes
32: materials ==> 60 vezes
33: equação ==> 60 vezes
34: cálculo ==> 60 vezes
35: sua ==> 59 vezes
36: radiação ==> 59 vezes
37: papel ==> 59 vezes
38: básica ==> 59 vezes
39: terra ==> 58 vezes
40

Gerando vetores para palavras frequentes. Aqui, me refiro a vetores, porque são espécies de histogramas indexados pela própria palavra. Python permite esse tipo de estrutura através do tipo "dicionário", ou dict. Assim, é criado um dicionário que contém cada palavra da ementa como chave e o número de ocorrências como entrada. Ex. se a palavra civilização occorre 3 vezes, teremos uma linha do dicionário que será V['civilização']=3, ou {'civilização':3}. As duas maneiras são idênticas para o Python.

Uma vez criado o vetor de todas as palavras, de todas as ementas, criamos um vetor para cada disciplina, usando como base o vetor geral, de forma que o dicionário de todas as ementas são iguais no número de entradas e nas chaves, somente diferindo no número de ocorrência de cada palavra. 

Fazendo os vetores idênticos, podemos criar uma matriz "empilhando" os vetores somente do número de entradas. Com isso, criamos uma matriz onde cada linha é o vetor de cada ementa do catálogo. As entradas da matriz V[i,j] são o número de occorrências de palavra[j] na ementa[i], para j indo da primeira à última palavra de todo o catálogo e i indo de 1 até o número de disciplinas.

In [16]:
V = np.zeros((len(catalogo), len(emptyPairs)),dtype=int)    # inicia o vetor com o tamanho adequado (número de ementas)
l = len(emptyPairs)                                         # guarda o valor do número de palavras total do catálogo
palavras = list()
palavras.append('none')
for k in range(1,len(catalogo)):                           # loop para cada disciplina do catálogo
    start = time.time()
    if k%10==0:
        print(k)
        
    estaSigla  = catalogo[k][colSigla ]                     # guarda a sigla da disciplina como uma string
    estaEmenta = catalogo[k][colEmenta] + ' ' + catalogo[k][colBibliB]
    
    
    estaEmentaLimpa = limpaTexto(estaEmenta,stopWords)      # remove as palavras muito frequêntes como preposições, etc
    
        
    
    palavras.append(estaEmentaLimpa.split())                # cria lista com as palavras menos frequentes de cada ementa
    
    print('append '+str(time.time()-start),end=' ')
    
    
    esteVetor = criaVetor(estaEmentaLimpa)                  # cria o vetor com a contagem das palavras para essa disc.
    
    print('criaVetor '+str(time.time()-start),end=' ')
    
    
    if ELIM_MULT_OCORRENCIAS:
        for p in esteVetor.keys():                          # elimina múltiplas contagens de uma mesma palavra
            if esteVetor[p]>0:                              # deixando o vetor somente com entradas 0 ou 1
                esteVetor[p]=1
    print('ELIM_MUL '+str(time.time()-start),end=' ')
    
    vetorCompleto = emptyPairs.copy()                       # cria uma cópia do histograma de todo o catálogo
    vetorCompleto.update(esteVetor)                         # joga as contagens das palavras dessa disciplina no 
                                                            # dicionário geral. Esse passo é necessário para deixar todos
                                                            # os dicionários das disciplinas com o mesmo tamamho e na
                                                            # ordem.
    print('vertorCompleto '+str(time.time()-start),end=' ')
    
    
    if len(vetorCompleto) != l:                             # Aqui é um pequeno bug. Quando uma ementa começa com uma
                                                            # palavra frequente, o algoritmo náo consegue remover
                                                            # então preciso fazer essa checagem para uniformizar os vetores
        s1 = set(vetorCompleto.keys())                      # joga todas as palavras dessa disciplina em um conjunto (set)
        s2 = set(allPairs.keys())                           # joga todas as palavras de todas as disciplinas em um set
        s1.difference_update(s2)                            # identifica qual é a palavra diferente guarda em s1
        
        for aux in s1:                                     # for para todas essas palavras
            del vetorCompleto[aux]                          # apaga as entradas do dicionário dessa disciplina 
    #print(vetorCompleto.values())
    type(vetorCompleto)
    
    V[k][:] = np.fromiter(iter(vetorCompleto.values()),dtype=int) # finalmente cria o vetor para essa disciplina e guarda em uma
                                                                  # linha da matriz
    print('V[k][:] '+str(time.time()-start),end=' ')
     
    M = np.inner(V,V)           # multiplica a matriz V pela transposta (V'), de forma a obter
                                # um produto escalar dos histogramas, que dão uma medida da 
                                # da sobreposição entre eles.

append 0.0002319812774658203 criaVetor 0.0008690357208251953 ELIM_MUL 0.0010113716125488281 vertorCompleto 0.0018055438995361328 V[k][:] 0.0024344921112060547 append 0.00017142295837402344 criaVetor 0.0004031658172607422 ELIM_MUL 0.00043964385986328125 vertorCompleto 0.001672506332397461 V[k][:] 0.002330303192138672 append 0.0002715587615966797 criaVetor 0.0007300376892089844 ELIM_MUL 0.0007739067077636719 vertorCompleto 0.0018999576568603516 V[k][:] 0.0025587081909179688 append 0.000171661376953125 criaVetor 0.00041174888610839844 ELIM_MUL 0.0004513263702392578 vertorCompleto 0.0014486312866210938 V[k][:] 0.0021741390228271484 append 0.00019693374633789062 criaVetor 0.0005137920379638672 ELIM_MUL 0.0005552768707275391 vertorCompleto 0.0016012191772460938 V[k][:] 0.0021309852600097656 append 0.00017571449279785156 criaVetor 0.0004551410675048828 ELIM_MUL 0.0004947185516357422 vertorCompleto 0.0018687248229980469 V[k][:] 0.002504587173461914 append 0.00015616416931152344 criaVetor 0.000

Nesse ponto, temos uma matriz simétrica M[i,j] onde cada entrada é o produto escalar entre a disciplina[i] e a displina[j]. Porém o produto escalar pode variar muito com o tamanho das ementas. Assim, uma medida melhor é dividir o produto escalar pela "norma" de cada disciplina comparada, ou seja, criar um coeficiente coef = M[i,j]/(M[i,i]*M[j,j]), de forma que o coef tenha um valor máximo de 1 (100%) quando as ementas forem idênticas, e zero quando não tiverem qualquer palavra em comum.

#Ordenando por sobreposição
Aqui é somente uma preciosidade de ordenar as disciplinas por sobreposição, das mais sobrepostas às menos sobrepostas.

In [17]:
(I,J) = M.nonzero()   # Busca por valores não nulos na matriz M e joga os índices em I e J

aux = np.array([[I[k],J[k],float(M[I[k],J[k]]*M[I[k],J[k]])/float(M[I[k],I[k]]*M[J[k],J[k]])] for k in range(I.size) ])
# Deixando o código abaixo, caso se queira ordenar pelo coeficiente 2 (coef2)
#minimo = min(M[I[k],I[k]],M[J[k],J[k]])
#coef2= float(M[I[k],J[k]])/float(minimo)
#aux = np.array([[I[k],J[k],coef2] for k in range(I.size) ])

aux = aux[aux[:,2].argsort(),]             # ordena o vetor
aux = aux[::-1,]                           # coloca o vetor em ordem reversa (de maior sobreposição para menor)

I = aux[0:,0].tolist()                     # converte os índices, agora ordenados para uma lista do python
I = [int(i) for i in I]                    # converte a lista para uma lista de inteiros
J = aux[0:,1].tolist()                     # converte os índicer, agora ordenados para uma lista do python
J = [int(j) for j in J]                    # converte a lista para uma lista de inteiros

Nesse ponto temos os índices I e J definem os índices das disciplinas que têm alguma sobreposição. As listas estão organizadas da maior sobre posição para a menor.

# Gerando lista com ementas em ordem de semelhança
* Observação:foram eliminadas as disciplinas que contém as palavras: estágio, trabalho, tcc etc (ver código abaixo). Isso é para eliminar as disciplinas como trabalho de graduação

In [18]:
for k in range(len(I)):                                # loop para cada disciplina

    # --- Calculando o coeficiente de sobreposição ---
    coef = float(M[I[k],J[k]]*M[I[k],J[k]])/float(M[I[k],I[k]]*M[J[k],J[k]])
    minimo = min(M[I[k],I[k]],M[J[k],J[k]]);
    coef2= float(M[I[k],J[k]])/float(minimo)
    nome = catalogo[I[k]][colNome].lower().split()
    
    # --- separando as ementas em listas de palavras para comparação ---
    ementaI = catalogo[I[k]][colEmenta].split()
    ementaJ = catalogo[J[k]][colEmenta].split()
    
    # --- Visualisando as emenstas em ordem decrescente de similaridade ---
    #     Note que usamos I[k]<J[k]-3 para remover as que estão menos de 3 
    #     ementas de "distância" no catálogo, que normalmente são versões
    #     da mesma disciplina '''
    if 0.1 < coef < 2 and I[k] < J[k]-3 and 'graduação' not in set(nome) and  \
        'estágio' not in set(nome) and    'tcc'       not in set(nome): 
        
        # --- Imprimindo os coeficientes na tela para informação ---
        print('Sobreposição = ',int(round(coef*100)),'%\t', 'Sobreposição 2 = ', int(round(coef2*100)),'%')
        
        # --- Encontrando e imprimindo as palavras que estão em ambas as ementas --- 
        print('Palavras em comum:')
        palavrasComuns = set(palavras[I[k]]).intersection(set(palavras[J[k]]))  # encontra intersecção entre ementas
        palavrasComuns = list(palavrasComuns)                                   # transforma em uma lista
        
        for i in range(len(palavrasComuns)):
            print(palavrasComuns[i],end=' ')      # imprime palavras encontradas em ambas as disciplinas, na mesma linha
            
        # --- imprimindo as ementas na tela para comparação ---    
        print('\n')
        print(catalogo[I[k]][0],'-', catalogo[I[k]][1], '  --- número', I[k], ' do catálogo')
        print(catalogo[I[k]][colEmenta])
        print(catalogo[I[k]][colBibliB])
        
        print('\n')
        print(catalogo[J[k]][0],'-', catalogo[J[k]][1], '  --- número', J[k], ' do catálogo')
        print(catalogo[J[k]][colEmenta])
        print(catalogo[J[k]][colBibliB])
        
        print('\n','_________________________________________','\n\n') 

Sobreposição =  64 %	 Sobreposição 2 =  90 %
Palavras em comum:
básicos porto 7a nelson p revinter cox 580 imunologia celular walport michael k distúrbios 5 sarvier 2005 artmed funções roitt imune shiv abbas c 6 conceitos básica bioquímica princípios ed david 307 pill alegre 2010 male imunobiologia j 4 edição janeway h travers m 7 l i andrew lichtman paulo são brostoff 2003 sistema 2006 abul manole albert molecular rio elsevier 2012 lehninger janeiro 1202 

NHT1055-15 - Fundamentos de Imunologia   --- número 411  do catálogo
Origem, evolução e conceitos básicos da imunidade inata e adquirida. Identificação dos componentes moleculares e celulares das repostas inata e adquirida do sistema imunológico. Reconhecimento dos órgãos e tecidos associados ao desenvolvimento e amadurecimento das células envolvidas na imunidade inata e adquirida. Mecanismos moleculares da geração da diversidade dos receptores envolvidos na resposta imunológica adquirida.
ABBAS, Abul K., LICHTMAN, Andrew H., SHIV P